## Making requests


Let's start with a minimal request.
It queries the title of every incident in the database.

In [1]:
import requests
minimal_example_response = requests.post(
    'https://incidentdatabase.ai/api/graphql',
    data='{"query": "query { incidents { title } }"}',
    headers={"content-type": "application/json"}
)
minimal_example_response

<Response [200]>

In [2]:
minimal_example_response.json()['data']['incidents'][0:5]

[{'title': 'Las Vegas Self-Driving Bus Involved in Accident'},
 {'title': 'Uber AV Killed Pedestrian in Arizona'},
 {'title': 'Google’s YouTube Kids App Presents Inappropriate Content'},
 {'title': 'Gender Biases of Google Image Search'},
 {'title': 'Common Biases of Vector Embeddings'}]

We can put some of the scaffolding in a function.

In [3]:
import requests
import json

def post_query(query, variables={}):
    response = requests.post(
        'https://incidentdatabase.ai/api/graphql',
        data=json.dumps({ "query": query, "variables": variables }),
        headers={"content-type": "application/json"}
    )
    response_json = response.json()
    if response_json.get('errors'):
        raise Exception(response_json['errors'])
    return response_json['data']

In [4]:
post_query("query { incidents { title, incident_id } }")['incidents'][0:5]

[{'title': 'Las Vegas Self-Driving Bus Involved in Accident',
  'incident_id': 23},
 {'title': 'Uber AV Killed Pedestrian in Arizona', 'incident_id': 4},
 {'title': 'Google’s YouTube Kids App Presents Inappropriate Content',
  'incident_id': 1},
 {'title': 'Gender Biases of Google Image Search', 'incident_id': 18},
 {'title': 'Common Biases of Vector Embeddings', 'incident_id': 12}]

## Pagination and Using Variables

So far we've been requesting data for every incident in the database.
This will become infeasable as we request more data per field.
To handle this, we use pagination.

In [5]:
post_query("""
    query {
        incidents(pagination: {limit: 3}) {
            title
            date
            description
        }
    }
""")

{'incidents': [{'title': 'Las Vegas Self-Driving Bus Involved in Accident',
   'date': '2017-11-08',
   'description': 'A self-driving public shuttle by Keolis North America and Navya was involved in a collision with a human-driven delivery truck in Las Vegas, Nevada on its first day of service.'},
  {'title': 'Uber AV Killed Pedestrian in Arizona',
   'date': '2018-03-18',
   'description': 'An Uber autonomous vehicle (AV) in autonomous mode struck and killed a pedestrian in Tempe, Arizona.'},
  {'title': 'Google’s YouTube Kids App Presents Inappropriate Content',
   'date': '2015-05-19',
   'description': 'YouTube’s content filtering and recommendation algorithms exposed children to disturbing and inappropriate videos.'}]}

In [6]:
page = -1

In [7]:
page += 1
post_query(
    """
    query ($pagination: PaginationType) {
        incidents(pagination: $pagination) {
            title
            date
            description
        }
    }
    """,
    variables={"pagination": { "limit": 3, "skip": 3 * page }}
)

{'incidents': [{'title': 'Las Vegas Self-Driving Bus Involved in Accident',
   'date': '2017-11-08',
   'description': 'A self-driving public shuttle by Keolis North America and Navya was involved in a collision with a human-driven delivery truck in Las Vegas, Nevada on its first day of service.'},
  {'title': 'Uber AV Killed Pedestrian in Arizona',
   'date': '2018-03-18',
   'description': 'An Uber autonomous vehicle (AV) in autonomous mode struck and killed a pedestrian in Tempe, Arizona.'},
  {'title': 'Google’s YouTube Kids App Presents Inappropriate Content',
   'date': '2015-05-19',
   'description': 'YouTube’s content filtering and recommendation algorithms exposed children to disturbing and inappropriate videos.'}]}

Every time you run the above cell, it will produce the next "page" of 3 incidents.

## Incidents and Reports

Incidents in the AI Incident Database have one or more associated reports documenting them.
These can be queried within an incident query.

In [8]:
data = post_query(
    """
    query {
        incidents(pagination: { limit: 1 }) {
            title
            date
            description
            reports {
                title
                url
            }
        }
    }
    """
)
print(data['incidents'][0]['reports'][0:3])

[{'title': 'Self-driving shuttle bus in crash on first day', 'url': 'https://www.bbc.com/news/technology-41923814'}, {'title': 'The real lesson of that self-driving shuttle’s first-day accident', 'url': 'https://arstechnica.com/cars/2017/11/the-real-lesson-of-that-self-driving-shuttles-first-day-accident/'}, {'title': 'U.S. safety board to probe self-driving shuttle crash in Las Vegas', 'url': 'https://www.reuters.com/article/us-autos-selfdriving/u-s-safety-board-to-probe-self-driving-shuttle-crash-in-las-vegas-idUSKBN1DA2K1'}]


Reports are a top-level collection, so they can also be queried independent of incidents, with pagination.

In [9]:
post_query(
    """
    query {
        reports(pagination: { limit: 5 }) {
            title
            url
        }
    }
    """
)

{'reports': [{'title': 'Google’s YouTube Kids App Criticized for ‘Inappropriate Content’',
   'url': 'https://blogs.wsj.com/digits/2015/05/19/googles-youtube-kids-app-criticized-for-inappropriate-content/'},
  {'title': 'YouTube Kids, Criticized for Content, Introduces New Parental Controls',
   'url': 'https://www.nytimes.com/2018/04/25/business/media/youtube-kids-parental-controls.html'},
  {'title': "Children's YouTube is still churning out blood, suicide and cannibalism",
   'url': 'https://www.wired.co.uk/article/youtube-for-kids-videos-problems-algorithm-recommend'},
  {'title': 'YouTube Kids app is STILL showing disturbing videos',
   'url': 'https://www.dailymail.co.uk/sciencetech/article-5358365/YouTube-Kids-app-showing-disturbing-videos.html'},
  {'title': 'YouTube suggested conspiracy videos to children using its Kids app',
   'url': 'https://www.businessinsider.com/youtube-suggested-conspiracy-videos-to-children-using-its-kids-app-2018-3'}]}

## Filters

In [10]:
post_query(
    """
    query {
        reports(
            pagination: { limit: 5 }, 
            filter: { date_published: { GTE: "2024-01-01T00:00:00+00:00" }}
        ) {
            title
            url
            date_published
        }
    }
    """
)

{'reports': [{'title': 'Dealership AI Chatbot Sells Car for $1',
   'url': 'https://www.colombiaone.com/2024/01/08/gm-dealership-chatbot-ai/',
   'date_published': '2024-01-08T00:00:00.000Z'},
  {'title': 'Lazy use of AI leads to Amazon products called “I cannot fulfill that request”',
   'url': 'https://arstechnica.com/ai/2024/01/lazy-use-of-ai-leads-to-amazon-products-called-i-cannot-fulfill-that-request/',
   'date_published': '2024-01-12T00:00:00.000Z'},
  {'title': 'FTC Announces Groundbreaking Action Against Rite Aid for Unfair Use of AI',
   'url': 'https://www.wilmerhale.com/en/insights/blogs/wilmerhale-privacy-and-cybersecurity-law/20240111-ftc-announces-groundbreaking-action-against-rite-aid-for-unfair-use-of-ai',
   'date_published': '2024-01-11T00:00:00.000Z'},
  {'title': 'Rite Aid Settles FTC Allegations Regarding Use of Facial Recognition Technology',
   'url': 'https://www.insideprivacy.com/artificial-intelligence/rite-aid-settles-ftc-allegations-regarding-use-of-facial

In [11]:
post_query(
    """
    query {
        reports(
            pagination: { limit: 5 }, 
            filter: { 
                date_published: {
                    GTE: "2022-01-01T00:00:00+00:00",
                    LTE: "2023-01-01T00:00:00+00:00"
                },
                title: { REGEX: "ChatGPT" }
            }
        ) {
            title
            url
            date_published
        }
    }
    """
)

{'reports': [{'title': 'Temporary policy: ChatGPT is banned',
   'url': 'https://meta.stackoverflow.com/questions/421831/temporary-policy-chatgpt-is-banned',
   'date_published': '2022-12-05T00:00:00.000Z'},
  {'title': 'ChatGPT proves that AI still has a racism problem',
   'url': 'https://www.newstatesman.com/quickfire/2022/12/chatgpt-shows-ai-racism-problem',
   'date_published': '2022-12-10T00:00:00.000Z'},
  {'title': 'ChatGPT bot tricked into giving bomb-making instructions, say developers',
   'url': 'https://www.thetimes.co.uk/article/chatgpt-bot-tricked-into-giving-bomb-making-instructions-say-developers-rvktrxqb5',
   'date_published': '2022-12-17T00:00:00.000Z'},
  {'title': "Testing Ways to Bypass ChatGPT's Safety Features",
   'url': 'https://www.lesswrong.com/posts/7fYxxtZqjuYXhBA2D/testing-ways-to-bypass-chatgpt-s-safety-features',
   'date_published': '2022-12-06T00:00:00.000Z'},
  {'title': 'ChatGPT is being used to create malicious emails and code',
   'url': 'https:/

## Comparing count of articles with given keywords over time

In [12]:
keyword_in_date_range_query = """
query ($keyword: String, $published_gte: DateTime, $published_lt: DateTime) {
  reports(
    filter: {
      plain_text: { REGEX: $keyword },
      date_published: {
        GTE: $published_gte,
        LT: $published_lt
      }
    }
  ) {
    title
  }
}
"""

In [13]:
ai_2020_2024 = post_query(
    keyword_in_date_range_query, 
    variables={
        'keyword': "\\W(?i)ai\\W",
        'published_gte': "2020-01-01T00:00:00+00:00",
        'published_lt': "2024-01-01T00:00:00+00:00",
    }
)
print(len(ai_2020_2024['reports']))

1126


In [14]:
for year in range(2008, 2024, 4):
    lower = str(year)
    upper = str(year + 4)
    print(lower + "-" + upper)
    for keyword in "artificial intelligence", "machine learning":
        data = post_query(
            keyword_in_date_range_query, 
            variables={
                'keyword': "\\W(?i)" + keyword +"\\W",
                'published_gte': lower + "-01-01T00:00:00+00:00",
                'published_lt': upper + "-01-01T00:00:00+00:00",
            }
        )
        print('  ', keyword, len(data['reports']))
    print() 

2008-2012
   artificial intelligence 0
   machine learning 0

2012-2016
   artificial intelligence 15
   machine learning 11

2016-2020
   artificial intelligence 226
   machine learning 150

2020-2024
   artificial intelligence 701
   machine learning 167

